In [1]:
from torch import torch
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt


In [2]:

train_data = datasets.MNIST("../", train=True, download=True)
test_data = datasets.MNIST("../", train=False, download=True)

x_train = train_data.data.reshape(-1, 784).float() / 255
x_test = test_data.data.reshape(-1, 784).float() / 255
y_train = train_data.targets
y_test = test_data.targets


In [42]:
loss_func = F.cross_entropy

def accuracy(input, target):
    preds = torch.argmax(input, dim=1)
    return (preds == target).float().mean()

In [43]:
class Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)
        
    def forward(self, x):
        return self.lin(x)


In [44]:
batch_size = 64
x_batch = x_train[0:batch_size]
y_batch = y_train[0:batch_size]

model = Logistic()
preds = model.forward(x_batch)
loss = loss_func(preds, y_batch)
print(loss)
print(accuracy(preds, y_batch))

tensor(2.2695, grad_fn=<NllLossBackward0>)
tensor(0.1562)


In [45]:
from ast import mod


lr = 0.5
epochs = 2

for epoch in range(epochs):
    # run n // batch_size times if n == k * batch_size
    # run n // batch_size + 1 times if n != k * batch_size
    n = x_train.shape[0]
    for i in range((n - 1) // batch_size + 1):
        index = slice(i * batch_size, (i + 1) * batch_size)
        x_batch = x_train[index]
        y_batch = y_train[index]
        preds = model.forward(x_batch)
        loss = loss_func(preds, y_batch)
        print(loss)
        print(accuracy(preds, y_batch))

        loss.backward()
        with torch.no_grad():
            for param in model.parameters():
                param -= lr * param.grad
            model.zero_grad()

tensor(2.2695, grad_fn=<NllLossBackward0>)
tensor(0.1562)
tensor(1.7965, grad_fn=<NllLossBackward0>)
tensor(0.3125)
tensor(1.8243, grad_fn=<NllLossBackward0>)
tensor(0.3281)
tensor(1.5411, grad_fn=<NllLossBackward0>)
tensor(0.5000)
tensor(1.2709, grad_fn=<NllLossBackward0>)
tensor(0.6406)
tensor(0.9660, grad_fn=<NllLossBackward0>)
tensor(0.7812)
tensor(0.8847, grad_fn=<NllLossBackward0>)
tensor(0.8125)
tensor(1.0723, grad_fn=<NllLossBackward0>)
tensor(0.7344)
tensor(0.9770, grad_fn=<NllLossBackward0>)
tensor(0.7500)
tensor(1.1440, grad_fn=<NllLossBackward0>)
tensor(0.6562)
tensor(0.9341, grad_fn=<NllLossBackward0>)
tensor(0.7500)
tensor(0.9573, grad_fn=<NllLossBackward0>)
tensor(0.6875)
tensor(0.8410, grad_fn=<NllLossBackward0>)
tensor(0.6562)
tensor(0.9456, grad_fn=<NllLossBackward0>)
tensor(0.6406)
tensor(1.1097, grad_fn=<NllLossBackward0>)
tensor(0.6562)
tensor(0.7403, grad_fn=<NllLossBackward0>)
tensor(0.8281)
tensor(0.9800, grad_fn=<NllLossBackward0>)
tensor(0.7344)
tensor(0.9691,

In [41]:
print(loss_func(model(x_batch), y_batch), accuracy(model(x_batch), y_batch))

tensor(0.0525, grad_fn=<NllLossBackward0>) tensor(1.)
